In [2]:

import numpy as np
import matplotlib.pyplot as plt
import os 
import cv2 

DataDir="F:/2022_Summer/test2data"
Categories=["Post-Injury", "Pre-Injury"]

trainingdata=[] 
def createtrainingdata():
#creates a data set of images and labels 
    print("Training data is loading...")
    premat=[]
    postmat=[]
    for category in Categories: 
        path=os.path.join(DataDir,category)
        classnum=Categories.index(category)
        for vid in os.listdir(path):
            vpath=os.path.join(path,vid)
            j=0
            for frame in os.listdir(vpath): 
                img_array=cv2.imread(os.path.join(vpath,frame),0)
                #if j==0: 
                    #vid_array=img_array
                #else:
                    #vid_array=np.concatenate([vid_array,img_array])
                if classnum ==0:
                    postmat.append(img_array)
                else:
                    premat.append(img_array)
                #j=j+1
                trainingdata.append([img_array,classnum])
    
    postav=np.average(np.array(postmat))
    #print(postav)
    preav=np.average(np.array(premat))
    #print(preav)
 
    
    #Centers the data set to control for the lighting and improve processing speed 
    print("Training data is being centered...")
    for d in trainingdata: 
        if d[1]==1:
            d[0]=d[0]-preav
        else:
            d[0]=d[0]-postav
    
                           
    print("Done!")
createtrainingdata()

Training data is loading...
Training data is being centered...
Done!


In [3]:
#Randomizes the order of the data set 
import random 
random.shuffle(trainingdata)
print("Data is randomized!")

Data is randomized!


In [4]:
#Changes the trainingdata into X and Y numpy arrays to be inputted into the model 
X=[]
Y=[]
for features,label in trainingdata:
    X.append(features)
    Y.append(label)
X=np.array(X)
Y=np.array(Y)
print(X.shape)
print(X[1].shape)
X=X.reshape(-1,X[1].shape[0],X[1].shape[1],1)

print("Data is formatted!")

(4160, 425, 425)
(425, 425)
Data is formatted!


In [5]:
#saves training data 
import pickle 
pickle_out=open("X.pickle", "wb")
pickle.dump(X,pickle_out)
pickle_out.close()
pickle_out=open("Y.pickle", "wb")
pickle.dump(Y,pickle_out)
pickle_out.close()
print("Data is saved!")

Data is saved!


In [7]:

import tensorflow as tf 
import pickle 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time 


    
#NAME="Pre-vs-Post-Injury-CNN-{}".format(int(time.time()))
NAME="Middle(10%),5_Images{}_SGD_Optimizer".format(int(time.time()))
tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/{}'.format(NAME), histogram_freq=1, profile_batch = 0
)
X=pickle.load(open("X.pickle", "rb"))
Y=pickle.load(open("Y.pickle", "rb"))
#Normalizes the values 
X=X/255.0

model=Sequential()
model.add(Conv2D(64, (3,3),input_shape=(425, 425, 1)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3,3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dense(1))

model.add(Activation('sigmoid'))

model.compile(loss="binary_crossentropy", optimizer="SGD", metrics=['accuracy'])

model.fit(X,Y, batch_size=32,epochs=8,validation_split=0.2,callbacks=[tensorboard, tensorboard_callback])



Epoch 1/8
104/104 [==============================] - 11s 107ms/step - loss: 0.4493 - accuracy: 0.8053 - val_loss: 0.3704 - val_accuracy: 0.7993
Epoch 2/8
104/104 [==============================] - 11s 101ms/step - loss: 0.2634 - accuracy: 0.9005 - val_loss: 0.3516 - val_accuracy: 0.8558
Epoch 3/8
104/104 [==============================] - 10s 101ms/step - loss: 0.1589 - accuracy: 0.9453 - val_loss: 0.1174 - val_accuracy: 0.9700
Epoch 4/8
104/104 [==============================] - 10s 100ms/step - loss: 0.0974 - accuracy: 0.9715 - val_loss: 0.0987 - val_accuracy: 0.9808
Epoch 5/8
104/104 [==============================] - 10s 100ms/step - loss: 0.0465 - accuracy: 0.9895 - val_loss: 0.0703 - val_accuracy: 0.9904
Epoch 6/8
104/104 [==============================] - 10s 100ms/step - loss: 0.0893 - accuracy: 0.9781 - val_loss: 0.0605 - val_accuracy: 0.9856
Epoch 7/8
104/104 [==============================] - 10s 100ms/step - loss: 0.0271 - accuracy: 0.9943 - val_loss: 0.0248 - val_accuracy:

ValueError: in user code:

    File "C:\Users\DeepLabCut\anaconda3\envs\vidan\lib\site-packages\keras\engine\training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\DeepLabCut\anaconda3\envs\vidan\lib\site-packages\keras\engine\training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\DeepLabCut\anaconda3\envs\vidan\lib\site-packages\keras\engine\training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\DeepLabCut\anaconda3\envs\vidan\lib\site-packages\keras\engine\training.py", line 1791, in predict_step
        return self(x, training=False)
    File "C:\Users\DeepLabCut\anaconda3\envs\vidan\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None

    ValueError: Exception encountered when calling layer "conv2d_2" (type Conv2D).
    